In [1]:
import json
import re

def format_exp(s):
    # Define a function to format a match
    bad_desc = ""
    def format_match(match):
        # Split the numbers by comma and strip any spaces
        numbers = [num.strip() for num in match.group(0)[1:-1].split(',')]
        try:
            numbers[1] = f"{float(numbers[1]):.2f}"
        except:
            pass
        numbers[2] = f"{float(numbers[2]):.2f}"
        # Reconstruct the string with the formatted numbers
        nonlocal bad_desc
        bad_desc += f"The {numbers[0]} feature has a values of {numbers[1]} and a SHAP value of {numbers[2]}. "
        return f"({', '.join(numbers[0:3])})"
    
    # Find all sets of numbers within parentheses
    pattern = re.compile(r'\([^()]*\)')
    result = re.sub(pattern, format_match, s)
    return result, bad_desc

def parse_convo(convo):
  exp = convo['messages'][1]['content']
  exp, bad_desc = format_exp(exp)
  
  desc = convo['messages'][2]['content']
  
  return {"explanation":exp, 
          "context":"The ML model predicts house prices",
          "explanation_format":"SHAP feature contribution in (feature_name, feature_value, contribution) format",
          "description":desc,
          "bad_description":bad_desc}

training_data = json.load(open("finetuning_data.json", "r"))
examples = []
for convo in training_data:
  examples.append(parse_convo(convo))



In [2]:
# Add unlabeled examples
import os
import pandas as pd

def add_examples(filenames, context):
    def parse_exp(exp, num_features=5):
        features = []
        if num_features is None:
            num_features = len(exp)
        for i in range(num_features):
            feature_name = exp[i]["Feature Name"].strip()
            feature_value = exp[i]["Feature Value"].strip()
            try:
                feature_value = f"{float(feature_value):.2f}"
            except:
                pass
            contribution = f"{float(exp[i]['Contribution']):.2f}"

            features.append(f"({feature_name}, {feature_value}, {contribution})")
        return ", ".join(features)
    
    exp_files = [
        os.path.join("..", "data", f"{filenames}_{i}.csv") for i in range(3)
    ]
    exps = []
    for exp in exp_files:
        exp_df = pd.read_csv(exp)
        exp_df = exp_df.sort_values(by="Contribution", key=abs, ascending=False)
        exps.append(exp_df.to_dict("records"))
    
    for exp in exps:
        examples.append(
            {
                "explanation": parse_exp(exp),
                "context": context,
                "explanation_format": "SHAP feature contribution in (feature_name, feature_value, contribution) format",
            }
        )
    
        
add_examples("ames_housing", "The ML model predicts house prices")
add_examples("mushroom", "The model predicts whether a mushroom is poisonous")
add_examples("pdf_benign", "The model predicts whether a PDF file contains malware")
add_examples("pdf_malware", "The model predicts whether a PDF file contains malware")
add_examples("student_performance", "The model predicts if a student with pass their class")

In [3]:
with open("examples.json", "w") as f:
  json.dump(examples, f, indent=2)